In [1]:
import sqlite3
from collections import Counter
import pandas as pd
import nltk
import plotly.express as px

In [2]:
query = 'select * from daily_top20_tracks;'
query_artists = 'select * from artists;'

In [3]:
conn = sqlite3.connect('spotify.db')
cursor = conn.cursor()

In [4]:
cursor.execute(query)
blob = cursor.fetchall()

In [5]:
genres_rawcount = [i[4] for i in blob]

In [6]:
genre_blob = ' '.join(genres_rawcount)

In [7]:
genre_wordlist = nltk.word_tokenize(genre_blob)

In [8]:
#no NLTK
raw_grams = Counter(genres_rawcount).most_common()

#top 10 bigrams
bigrams = Counter(list(nltk.bigrams(genre_wordlist))).most_common(10)

#top 10 unigrams
unigrams = list(enumerate(Counter(genre_wordlist).most_common(10)))

## Chart Topper Lifespans

In [14]:
cols = {1:'position', 2:'art_id',3:'art_name',4:'album_name',5:'track_id',6:'track_name',7:'popularity',8:'query_date'}

In [15]:
df = pd.DataFrame(blob)
df.drop(0, axis=1, inplace=True)
df.rename(columns = cols, inplace=True)

In [16]:
number_1s = df.loc[df['position'] == '1']

In [46]:
number_1s['query_date'] = number_1s['query_date'].astype('datetime64[ns]')
number_1s['position'] = number_1s['position'].astype('int')

C:\Users\flamb\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\flamb\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [58]:
number_1s

,position,art_id,art_name,album_name,track_id,track_name,popularity,query_date
0,1,0xK5OqvQKGC1nNtl2pIbwX,Bob Log III,Guitar Party Power,7cpEBbFirsX4rq2zwqd96L,Ooo Ah Ooo Uh,11,2020-12-13
20,1,0xK5OqvQKGC1nNtl2pIbwX,Bob Log III,Guitar Party Power,7cpEBbFirsX4rq2zwqd96L,Ooo Ah Ooo Uh,11,2020-12-14
40,1,0xK5OqvQKGC1nNtl2pIbwX,Bob Log III,Guitar Party Power,7cpEBbFirsX4rq2zwqd96L,Ooo Ah Ooo Uh,11,2020-12-16
60,1,0xK5OqvQKGC1nNtl2pIbwX,Bob Log III,Guitar Party Power,7cpEBbFirsX4rq2zwqd96L,Ooo Ah Ooo Uh,11,2020-12-17
80,1,0xK5OqvQKGC1nNtl2pIbwX,Bob Log III,Guitar Party Power,7cpEBbFirsX4rq2zwqd96L,Ooo Ah Ooo Uh,11,2020-12-18
...,...,...,...,...,...,...,...,...
3000,1,4q0N3EI67tVnAeeaXbNQIj,J. Worra,Ride or Cry,1hKQtW2lzgBCSoUBf084lJ,Ride or Cry,23,2021-06-08
3020,1,4q0N3EI67tVnAeeaXbNQIj,J. Worra,Ride or Cry,1hKQtW2lzgBCSoUBf084lJ,Ride or Cry,23,2021-06-09
3040,1,4q0N3EI67tVnAeeaXbNQIj,J. Worra,Ride or Cry,1hKQtW2lzgBCSoUBf084lJ,Ride or Cry,23,2021-06-10
3060,1,4q0N3EI67tVnAeeaXbNQIj,J. Worra,Ride or Cry,1hKQtW2lzgBCSoUBf084lJ,Ride or Cry,23,2021-06-11


In [60]:
#find the time_delta between these. Plot them on a chart
number_1_timespans = number_1s.groupby(['art_name','track_name']).agg(['max','min'])['query_date']

In [68]:
(number_1_timespans['max'] - number_1_timespans['min']).sort_values(ascending=False)

art_name                 track_name                                       
100 gecs                 stupid horse (Remix) [feat. GFOTY & Count Baldor]   22 days
THICK                    Your Mom                                            19 days
Ponty Mython             Grooff Machine - Original Mix                       18 days
Mick Gordon              Rip & Tear                                          16 days
SLIFT                    Doppler Ganger                                      14 days
Octo Octa                Spin Girl, Let’s Activate!                          12 days
Skinny Girl Diet         Okay                                                12 days
J. Worra                 Ride or Cry                                         10 days
upper class              Conventional Parties                                 6 days
Nitepunk                 Flow (Habstrakt Remix)                               6 days
Bob Log III              Ooo Ah Ooo Uh                                     

# Latest Data date

In [51]:
number_1s.query_date.max()

Timestamp('2021-06-13 00:00:00')

---

In [47]:
blob = number_1s.groupby(['art_name','track_name','query_date']).sum()

In [56]:
blob.columns

Index(['position'], dtype='object')

In [54]:
blob.query_date.max()

AttributeError: 'DataFrame' object has no attribute 'query_date'

In [49]:
blob.groupby(['art_name','track_name']).sum().sort_values(by='position')

,,position
art_name,track_name,
Special Request,Spectral Frequency,1
Blind Melon,Three Is A Magic Number,1
Queens of the Stone Age,Give the Mule What He Wants,1
Potty Mouth,The Spins,1
The Knocks,Bodies,2
311,Self Esteem,3
Antarctigo Vespucci,I Drew You Once in Art Class,3
Chris Lake,Feel So Good,3
Friend Within,For You,4


In [26]:
number_1s.groupby(['art_name','track_name']).count().sort_values(by='position', ascending=False)

,,position,art_id,album_name,track_id,popularity,query_date
art_name,track_name,,,,,,
100 gecs,stupid horse (Remix) [feat. GFOTY & Count Baldor],20,20,20,20,20,20
THICK,Your Mom,17,17,17,17,17,17
Ponty Mython,Grooff Machine - Original Mix,15,15,15,15,15,15
Octo Octa,"Spin Girl, Let’s Activate!",13,13,13,13,13,13
SLIFT,Doppler Ganger,12,12,12,12,12,12
Mick Gordon,Rip & Tear,11,11,11,11,11,11
J. Worra,Ride or Cry,10,10,10,10,10,10
Skinny Girl Diet,Okay,10,10,10,10,10,10
upper class,Conventional Parties,7,7,7,7,7,7


## Plotly 

In [ ]:
#Raw_grams
df_raw = pd.DataFrame(raw_grams, columns = ['genre','count'])

blob = df_raw['genre'] == 'None'

df_raw = df_raw.loc[~blob]

In [ ]:
fig = px.bar(df_raw.iloc[:10], x='genre', y='count')
fig.show()

In [ ]:
#top 10 unigrams
unigrams = Counter(genre_wordlist).most_common(10)
df_uni = pd.DataFrame(unigrams, columns = ['genre','count'])

In [ ]:
fig = px.bar(df_uni, x='genre', y='count')
fig.show()

## Pandas

In [ ]:
cols = ['index', 'position', 'art_id', 'art_name', 'popularity', 'followers', 'day_of_chart']

In [ ]:
chart_appearances = df.groupby(['art_name'])['position'].count()

In [ ]:
chart_appearances.sort_values().tail(20)